# Поиск ассоциативных правил

#### Импорт библиотек для работы.

In [52]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np  
import pandas as pd  
from apyori import apriori

#### Прочитаем файл.

In [2]:
pluginfile = pd.read_csv('D:\\МИРЭА\\Машинное обучение\\Пр6\\supermarket.csv', encoding='windows-1251', sep='\t')  
pluginfile

,Номер чека,Товар
0,160698,"КЕТЧУПЫ, СОУСЫ, АДЖИКА"
1,160698,МАКАРОННЫЕ ИЗДЕЛИЯ
2,160698,ЧАЙ
3,160747,МАКАРОННЫЕ ИЗДЕЛИЯ
4,160747,МЕД
...,...,...
144,167465,ВАФЛИ
145,167465,СЫРЫ
146,167465,МЕД
147,167465,СУХАРИ


#### Преобразуем колонку с Товарами.

In [3]:
aggr_dict = {}
for numb, items in pluginfile.values:
    if numb not in aggr_dict:
        aggr_dict[numb] = []
    aggr_dict[numb] += [item.strip() for item in items.split(',')]

In [4]:
aggr_dict[160698]

['КЕТЧУПЫ', 'СОУСЫ', 'АДЖИКА', 'МАКАРОННЫЕ ИЗДЕЛИЯ', 'ЧАЙ']

#### Сымитируем работу grid-search для поиска разных наборов правил с разными параметрами поддержки, уверенности и минимальным размером.

In [5]:
train_set = list(aggr_dict.values())

In [55]:
apr_res_list = [
    (sup, conf, length, res)
    for sup in [0.001, 0.003, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
    for conf in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
    for length in range(2, 8)
    if (res := list(apriori(train_set, min_support=sup, min_confidence=conf, min_length=length)))
]

In [56]:
print("Количество получившихся вариаций: ", len(apr_res_list))

Количество получившихся вариаций:  726


#### Получим уникальное количество правил.

In [57]:
uniq_rul = sorted(list(set([len(apr_item[-1]) for apr_item in apr_res_list])))
print(len(uniq_rul))
print(*uniq_rul)

48
13 18 19 20 21 23 29 30 32 44 47 53 56 57 58 61 62 64 67 68 73 74 75 78 79 80 85 87 89 91 92 93 102 104 105 106 115 117 119 128 134 147 148 158 160 162 166 167


#### Выполним агрегацию всех вариаций по количеству получившихся правил.

In [58]:
def cnt_rul(apr, cnt: int):
    return [apr_item for apr_item in apr if len(apr_item[-1]) == cnt]

#### Выберем уникальные наборы.

In [59]:
def find_uniq(apr_list, cnt: int):
    apr = cnt_rul(apr_list, cnt)
    first = {item.items: item.support for item in apr[0][-1]}
    res = [apr[0][-1]]
    for i in range(len(apr)):
        if first != {item.items: item.support for item in apr[i][-1]}:
            res.append(apr[i][-1])
    return res

#### Рассмотрим результат, который возвращает алгоритм.

In [60]:
def rul_assoc(results):
    lhs = [tuple(res[2][0][0])[0] for res in results if res[2][0][0] and res[2][0][1]]
    rhs = [tuple(res[2][0][1])[0] for res in results if res[2][0][0] and res[2][0][1]]
    sup = [res[1] for res in results if res[2][0][0] and res[2][0][1]]
    conf = [res[2][0][2] for res in results if res[2][0][0] and res[2][0][1]]
    lifts = [res[2][0][3] for res in results if res[2][0][0] and res[2][0][1]]
    res = zip(lhs, rhs, sup, conf, lifts)
    return pd.DataFrame(res, columns = ["Left hand side", "Right hand side", "Support", "Confidence", "Lift"]).drop_duplicates()

In [61]:
def busket_assoc(results):
    items = [', '.join(res[0]) for res in results if res[2][0][0] and res[2][0][1]]
    lhs = [', '.join(res[2][0][0]) for res in results if res[2][0][0] and res[2][0][1]]
    rhs = [', '.join(res[2][0][1]) for res in results if res[2][0][0] and res[2][0][1]]
    sup = [res[1] for res in results if res[2][0][0] and res[2][0][1]]
    conf = [res[2][0][2] for res in results if res[2][0][0] and res[2][0][1]]
    lifts = [res[2][0][3] for res in results if res[2][0][0] and res[2][0][1]]
    res = zip(items, lhs, rhs, sup, conf, lifts)
    return pd.DataFrame(res, columns = ["Items", "Left hand side", "Right hand side", "Support", "Confidence", "Lift"]).drop_duplicates()

#### Посмотрим на правила, которые наиболее хорошо отработаны.

In [62]:
max_lift = {}
for uniq in uniq_rul:
    res = rul_assoc(find_uniq(apr_list=apr_res_list, cnt=uniq)[0])
    if res.shape[0] > 10:
        max_lift[uniq] = res.nlargest(n = 1, columns = "Lift").values[-1][-1]

#### Отсортируем словарь.

In [63]:
def sord_dict(input_dict: dict) -> dict:
    return {k: v for k, v in sorted(input_dict.items(), key=lambda item: item[1], reverse=True)}

In [64]:
sord_dict(max_lift)

{128: 4.4,
 134: 4.4,
 147: 4.4,
 148: 4.4,
 158: 4.4,
 160: 4.4,
 162: 4.4,
 166: 3.3846153846153846,
 167: 3.3846153846153846,
 74: 2.2564102564102564,
 80: 2.2564102564102564,
 87: 2.2564102564102564,
 91: 2.2564102564102564,
 92: 2.2564102564102564,
 93: 2.2564102564102564,
 102: 2.2564102564102564,
 104: 2.2564102564102564,
 105: 2.2564102564102564,
 115: 2.2564102564102564,
 117: 2.2564102564102564,
 44: 2.2448979591836737,
 47: 2.2448979591836737,
 53: 2.2448979591836737,
 56: 2.2448979591836737,
 57: 2.2448979591836737,
 58: 2.2448979591836737,
 61: 2.2448979591836737,
 62: 2.2448979591836737,
 67: 2.2448979591836737,
 68: 2.2448979591836737,
 73: 2.2448979591836737,
 78: 2.2448979591836737,
 79: 2.2448979591836737,
 85: 2.2448979591836737,
 106: 1.9130434782608698,
 119: 1.9130434782608698,
 20: 1.9130434782608696,
 23: 1.9130434782608696}

### Различные комбинации правил

In [65]:
def pr_par(cnt) -> None:
    sup, conf, length, _ = cnt_rul(apr_res_list, cnt)[0]
    print(f'Поддержка = {sup*100}%')
    print(f'Надежность = {conf*100}%')
    print(f'Длина = {length}')

#### Выведем правила для алгоритма, у которого наибольшее количество правил.

In [66]:
pr_par(167)

Поддержка = 0.1%
Надежность = 10.0%
Длина = 2


In [67]:
rul_167 = find_uniq(apr_list=apr_res_list, cnt=167)[0]
res_167b = busket_assoc(rul_167)
res_167b.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
23,"ВАФЛИ, ЧАЙ, КЕТЧУПЫ, АДЖИКА","ВАФЛИ, АДЖИКА","ЧАЙ, КЕТЧУПЫ",0.022727,1.0,3.384615
24,"ВАФЛИ, ЧАЙ, СОУСЫ, АДЖИКА","ВАФЛИ, АДЖИКА","ЧАЙ, СОУСЫ",0.022727,1.0,3.384615
33,"ВАФЛИ, ЧАЙ, СОУСЫ, КЕТЧУПЫ","ВАФЛИ, КЕТЧУПЫ","ЧАЙ, СОУСЫ",0.022727,1.0,3.384615
51,"СОУСЫ, АДЖИКА, ВАФЛИ, КЕТЧУПЫ, ЧАЙ","ВАФЛИ, АДЖИКА","ЧАЙ, СОУСЫ, КЕТЧУПЫ",0.022727,1.0,3.384615
8,"СУХАРИ, ВАФЛИ, МАКАРОННЫЕ ИЗДЕЛИЯ","ВАФЛИ, МАКАРОННЫЕ ИЗДЕЛИЯ",СУХАРИ,0.022727,1.0,3.142857
1,"ВАФЛИ, КЕТЧУПЫ, АДЖИКА","ВАФЛИ, АДЖИКА",КЕТЧУПЫ,0.022727,1.0,1.913043
2,"ВАФЛИ, СОУСЫ, АДЖИКА","ВАФЛИ, АДЖИКА",СОУСЫ,0.022727,1.0,1.913043
6,"ВАФЛИ, СОУСЫ, КЕТЧУПЫ","ВАФЛИ, КЕТЧУПЫ",СОУСЫ,0.022727,1.0,1.913043
22,"СОУСЫ, ВАФЛИ, КЕТЧУПЫ, АДЖИКА","ВАФЛИ, АДЖИКА","СОУСЫ, КЕТЧУПЫ",0.022727,1.0,1.913043
3,"ВАФЛИ, ЧАЙ, АДЖИКА","ВАФЛИ, АДЖИКА",ЧАЙ,0.022727,1.0,1.333333


In [68]:
res_167a = rul_assoc(rul_167)
res_167a.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Left hand side,Right hand side,Support,Confidence,Lift
23,ВАФЛИ,ЧАЙ,0.022727,1.000000,3.384615
8,ВАФЛИ,СУХАРИ,0.022727,1.000000,3.142857
1,ВАФЛИ,КЕТЧУПЫ,0.022727,1.000000,1.913043
2,ВАФЛИ,СОУСЫ,0.022727,1.000000,1.913043
3,ВАФЛИ,ЧАЙ,0.022727,1.000000,1.333333
48,СУХАРИ,ЧАЙ,0.022727,0.500000,1.692308
16,СУХАРИ,СЫРЫ,0.022727,0.500000,1.157895
17,СУХАРИ,ЧАЙ,0.022727,0.500000,0.666667
9,ВАФЛИ,СУХАРИ,0.090909,0.285714,1.795918
12,ВАФЛИ,СУХАРИ,0.068182,0.214286,1.885714


#### Выведем правила для алгоритма, у которого наибольшая метрика заинтересованности.

In [69]:
pr_par(128)

Поддержка = 0.1%
Надежность = 60.0%
Длина = 2


In [70]:
rul_128 = find_uniq(apr_list=apr_res_list, cnt=128)[0]
res_128b = busket_assoc(rul_128)
res_128b.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
121,"МЕД, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУПЫ, ЧАЙ,...","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, КЕТЧУПЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.400000
124,"МЕД, СОУСЫ, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, ЧАЙ, СЫРЫ","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, СОУСЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.400000
125,"МЕД, СОУСЫ, МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУПЫ, ЧАЙ, ...","СЫРЫ, МЕД, КЕТЧУПЫ","ЧАЙ, СОУСЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.400000
126,"МЕД, СОУСЫ, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУП...","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, СОУСЫ, КЕТЧУПЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.400000
52,"ВАФЛИ, ЧАЙ, КЕТЧУПЫ, АДЖИКА","ВАФЛИ, АДЖИКА","ЧАЙ, КЕТЧУПЫ",0.022727,1.0,3.384615
53,"ВАФЛИ, ЧАЙ, СОУСЫ, АДЖИКА","ВАФЛИ, АДЖИКА","ЧАЙ, СОУСЫ",0.022727,1.0,3.384615
74,"ВАФЛИ, ЧАЙ, СОУСЫ, КЕТЧУПЫ","ВАФЛИ, КЕТЧУПЫ","ЧАЙ, СОУСЫ",0.022727,1.0,3.384615
96,"СОУСЫ, АДЖИКА, ВАФЛИ, КЕТЧУПЫ, ЧАЙ","ВАФЛИ, АДЖИКА","ЧАЙ, СОУСЫ, КЕТЧУПЫ",0.022727,1.0,3.384615
105,"МЕД, АДЖИКА, КЕТЧУПЫ, ЧАЙ, СЫРЫ","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, КЕТЧУПЫ",0.045455,1.0,3.384615
109,"МЕД, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, ЧАЙ, СЫРЫ","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,3.384615


In [71]:
res_128a = rul_assoc(rul_128)
res_128a.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Left hand side,Right hand side,Support,Confidence,Lift
121,СЫРЫ,ЧАЙ,0.045455,1.0,4.400000
52,ВАФЛИ,ЧАЙ,0.022727,1.0,3.384615
105,СЫРЫ,ЧАЙ,0.045455,1.0,3.384615
118,МЕД,ЧАЙ,0.045455,1.0,3.384615
28,ВАФЛИ,СУХАРИ,0.022727,1.0,3.142857
98,СЫРЫ,КЕТЧУПЫ,0.045455,1.0,2.200000
107,СЫРЫ,СОУСЫ,0.045455,1.0,2.200000
0,АДЖИКА,КЕТЧУПЫ,0.522727,1.0,1.913043
2,АДЖИКА,СОУСЫ,0.522727,1.0,1.913043
6,КЕТЧУПЫ,СОУСЫ,0.522727,1.0,1.913043


#### Рассмотрим модель с меньшим количеством правил.

In [72]:
pr_par(74)

Поддержка = 7.5%
Надежность = 60.0%
Длина = 2


In [73]:
rul_74 = find_uniq(apr_list=apr_res_list, cnt=74)[0]
res_74b = busket_assoc(rul_74)
res_74b.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
0,"КЕТЧУПЫ, АДЖИКА",АДЖИКА,КЕТЧУПЫ,0.522727,1.0,1.913043
2,"СОУСЫ, АДЖИКА",АДЖИКА,СОУСЫ,0.522727,1.0,1.913043
6,"СОУСЫ, КЕТЧУПЫ",КЕТЧУПЫ,СОУСЫ,0.522727,1.0,1.913043
12,"МЕД, КЕТЧУПЫ, АДЖИКА","МЕД, АДЖИКА",КЕТЧУПЫ,0.204545,1.0,1.913043
13,"СОУСЫ, КЕТЧУПЫ, АДЖИКА",АДЖИКА,"СОУСЫ, КЕТЧУПЫ",0.522727,1.0,1.913043
14,"СЫРЫ, КЕТЧУПЫ, АДЖИКА","СЫРЫ, АДЖИКА",КЕТЧУПЫ,0.227273,1.0,1.913043
15,"ЧАЙ, КЕТЧУПЫ, АДЖИКА","ЧАЙ, АДЖИКА",КЕТЧУПЫ,0.295455,1.0,1.913043
20,"МЕД, СОУСЫ, АДЖИКА","МЕД, АДЖИКА",СОУСЫ,0.204545,1.0,1.913043
21,"СЫРЫ, СОУСЫ, АДЖИКА","СЫРЫ, АДЖИКА",СОУСЫ,0.227273,1.0,1.913043
22,"ЧАЙ, СОУСЫ, АДЖИКА","ЧАЙ, АДЖИКА",СОУСЫ,0.295455,1.0,1.913043


In [74]:
res_74a = rul_assoc(rul_74)
res_74a.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Left hand side,Right hand side,Support,Confidence,Lift
0,АДЖИКА,КЕТЧУПЫ,0.522727,1.0,1.913043
2,АДЖИКА,СОУСЫ,0.522727,1.0,1.913043
6,КЕТЧУПЫ,СОУСЫ,0.522727,1.0,1.913043
12,МЕД,КЕТЧУПЫ,0.204545,1.0,1.913043
14,СЫРЫ,КЕТЧУПЫ,0.227273,1.0,1.913043
15,ЧАЙ,КЕТЧУПЫ,0.295455,1.0,1.913043
20,МЕД,СОУСЫ,0.204545,1.0,1.913043
21,СЫРЫ,СОУСЫ,0.227273,1.0,1.913043
22,ЧАЙ,СОУСЫ,0.295455,1.0,1.913043
46,МЕД,КЕТЧУПЫ,0.113636,1.0,1.913043


#### Рассмотрим модель еще меньше.

In [75]:
pr_par(20)

Поддержка = 25.0%
Надежность = 60.0%
Длина = 2


In [76]:
rul_20 = find_uniq(apr_list=apr_res_list, cnt=20)[0]
res_20b = busket_assoc(rul_20)
res_20b.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
0,"КЕТЧУПЫ, АДЖИКА",АДЖИКА,КЕТЧУПЫ,0.522727,1.000000,1.913043
2,"СОУСЫ, АДЖИКА",АДЖИКА,СОУСЫ,0.522727,1.000000,1.913043
5,"СОУСЫ, КЕТЧУПЫ",КЕТЧУПЫ,СОУСЫ,0.522727,1.000000,1.913043
11,"СОУСЫ, КЕТЧУПЫ, АДЖИКА",АДЖИКА,"СОУСЫ, КЕТЧУПЫ",0.522727,1.000000,1.913043
12,"ЧАЙ, КЕТЧУПЫ, АДЖИКА","ЧАЙ, АДЖИКА",КЕТЧУПЫ,0.295455,1.000000,1.913043
14,"ЧАЙ, СОУСЫ, АДЖИКА","ЧАЙ, АДЖИКА",СОУСЫ,0.295455,1.000000,1.913043
16,"ЧАЙ, СОУСЫ, КЕТЧУПЫ","ЧАЙ, КЕТЧУПЫ",СОУСЫ,0.295455,1.000000,1.913043
18,"СОУСЫ, ЧАЙ, КЕТЧУПЫ, АДЖИКА","ЧАЙ, АДЖИКА","СОУСЫ, КЕТЧУПЫ",0.295455,1.000000,1.913043
3,"ВАФЛИ, ЧАЙ",ВАФЛИ,ЧАЙ,0.295455,0.928571,1.238095
8,"СУХАРИ, ЧАЙ",СУХАРИ,ЧАЙ,0.295455,0.928571,1.238095


In [77]:
res_20a = rul_assoc(rul_20)
res_20a.nlargest(n = 10, columns = ['Confidence', 'Lift'])

,Left hand side,Right hand side,Support,Confidence,Lift
0,АДЖИКА,КЕТЧУПЫ,0.522727,1.000000,1.913043
2,АДЖИКА,СОУСЫ,0.522727,1.000000,1.913043
5,КЕТЧУПЫ,СОУСЫ,0.522727,1.000000,1.913043
12,ЧАЙ,КЕТЧУПЫ,0.295455,1.000000,1.913043
14,ЧАЙ,СОУСЫ,0.295455,1.000000,1.913043
3,ВАФЛИ,ЧАЙ,0.295455,0.928571,1.238095
8,СУХАРИ,ЧАЙ,0.295455,0.928571,1.238095
10,АДЖИКА,КЕТЧУПЫ,0.454545,0.869565,1.913043
13,АДЖИКА,СОУСЫ,0.454545,0.869565,1.913043
15,КЕТЧУПЫ,СОУСЫ,0.454545,0.869565,1.913043


## Поиск ответов на вопросы

### Какой товар с наибольшей достоверностью берут с вафлями? 

In [79]:
res_128a.loc[res_128a['Left hand side'] == 'ВАФЛИ'].nlargest(n = 2, columns = ['Confidence', 'Lift'])

,Left hand side,Right hand side,Support,Confidence,Lift
52,ВАФЛИ,ЧАЙ,0.022727,1.0,3.384615
28,ВАФЛИ,СУХАРИ,0.022727,1.0,3.142857


### Человек взял мед и сыры, какой один из товаров он скорее всего не возьмёт? 

In [81]:
honey_cheese = []
for items in res_128b['Left hand side'].values:
    if 'МЕД' in items and 'СЫРЫ' in items:
        honey_cheese.append(items)

In [83]:
all_prod = set(res_128a['Right hand side']) | set(res_128a['Left hand side'])
print(all_products)

{'МЕД', 'СОУСЫ', 'СУХАРИ', 'АДЖИКА', 'МАКАРОННЫЕ ИЗДЕЛИЯ', 'ВАФЛИ', 'КЕТЧУПЫ', 'ЧАЙ', 'СЫРЫ'}


In [85]:
res_128hc = set([item for items in res_128b.values if items[1] in honey_cheese for item in items[2].split(', ')])
res_128hc

{'КЕТЧУПЫ', 'МАКАРОННЫЕ ИЗДЕЛИЯ', 'СОУСЫ', 'СУХАРИ', 'ЧАЙ'}

In [87]:
diff = all_products - res_128hc
print(diff)

{'СЫРЫ', 'ВАФЛИ', 'МЕД', 'АДЖИКА'}


#### Вероятнее всего, к корзине с Медом и Сырами покупатель не добавит такие товары как **Аджика** и **Вафли**.

### Назовите 5 самых популярных наборов товаров (в наборе может бытьодин или несколько товаров).

#### Сравним результаты, опираясь на метрику поддержки.

In [89]:
res_74b.nlargest(n = 5, columns = ['Support', 'Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
0,"КЕТЧУПЫ, АДЖИКА",АДЖИКА,КЕТЧУПЫ,0.522727,1.000000,1.913043
2,"СОУСЫ, АДЖИКА",АДЖИКА,СОУСЫ,0.522727,1.000000,1.913043
6,"СОУСЫ, КЕТЧУПЫ",КЕТЧУПЫ,СОУСЫ,0.522727,1.000000,1.913043
13,"СОУСЫ, КЕТЧУПЫ, АДЖИКА",АДЖИКА,"СОУСЫ, КЕТЧУПЫ",0.522727,1.000000,1.913043
11,"КЕТЧУПЫ, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ",АДЖИКА,"КЕТЧУПЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.454545,0.869565,1.913043


In [90]:
res_128b.nlargest(n = 5, columns = ['Support', 'Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
0,"КЕТЧУПЫ, АДЖИКА",АДЖИКА,КЕТЧУПЫ,0.522727,1.000000,1.913043
2,"СОУСЫ, АДЖИКА",АДЖИКА,СОУСЫ,0.522727,1.000000,1.913043
6,"СОУСЫ, КЕТЧУПЫ",КЕТЧУПЫ,СОУСЫ,0.522727,1.000000,1.913043
16,"СОУСЫ, КЕТЧУПЫ, АДЖИКА",АДЖИКА,"СОУСЫ, КЕТЧУПЫ",0.522727,1.000000,1.913043
14,"КЕТЧУПЫ, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ",АДЖИКА,"КЕТЧУПЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.454545,0.869565,1.913043


### Опишите 4-5 ассоциативных правил, полученных в ходе выполнения работы.

#### Выведем 4 самых сильных правила, ориентируясь на метрику заинтересованности.

In [95]:
res_128b.nlargest(n = 4, columns = ['Lift'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
121,"МЕД, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУПЫ, ЧАЙ,...","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, КЕТЧУПЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.4
124,"МЕД, СОУСЫ, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, ЧАЙ, СЫРЫ","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, СОУСЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.4
125,"МЕД, СОУСЫ, МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУПЫ, ЧАЙ, ...","СЫРЫ, МЕД, КЕТЧУПЫ","ЧАЙ, СОУСЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.4
126,"МЕД, СОУСЫ, АДЖИКА, МАКАРОННЫЕ ИЗДЕЛИЯ, КЕТЧУП...","СЫРЫ, МЕД, АДЖИКА","ЧАЙ, СОУСЫ, КЕТЧУПЫ, МАКАРОННЫЕ ИЗДЕЛИЯ",0.045455,1.0,4.4


#### Выведем 4 самых слабых правила, ориентируясь на метрику поддержки:

In [96]:
res_128b.nsmallest(n = 4, columns = ['Support'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
11,"ВАФЛИ, КЕТЧУПЫ, АДЖИКА","ВАФЛИ, АДЖИКА",КЕТЧУПЫ,0.022727,1.0,1.913043
12,"ВАФЛИ, СОУСЫ, АДЖИКА","ВАФЛИ, АДЖИКА",СОУСЫ,0.022727,1.0,1.913043
13,"ВАФЛИ, ЧАЙ, АДЖИКА","ВАФЛИ, АДЖИКА",ЧАЙ,0.022727,1.0,1.333333
26,"ВАФЛИ, СОУСЫ, КЕТЧУПЫ","ВАФЛИ, КЕТЧУПЫ",СОУСЫ,0.022727,1.0,1.913043


#### Посмотрим, с чем чаще всего берут сухари.

In [97]:
res_128a.loc[res_128a['Left hand side'] == 'СУХАРИ'].nlargest(n = 1, columns = ['Support', 'Confidence'])

,Left hand side,Right hand side,Support,Confidence,Lift
9,СУХАРИ,ЧАЙ,0.295455,0.928571,1.238095


#### Посмотрим с чем реже всего берут мед.

In [99]:
res_128a.loc[res_128a['Left hand side'] == 'МЕД'].nsmallest(n = 1, columns = ['Support', 'Confidence'])

,Left hand side,Right hand side,Support,Confidence,Lift
89,МЕД,СОУСЫ,0.045455,1.0,1.913043


#### Оценим самые невыгодные сочетания по метрике заинтересованности.

In [100]:
res_128b.nsmallest(n = 4, columns = ['Lift', 'Support'])

,Items,Left hand side,Right hand side,Support,Confidence,Lift
44,"МЕД, МАКАРОННЫЕ ИЗДЕЛИЯ, ЧАЙ","МЕД, МАКАРОННЫЕ ИЗДЕЛИЯ",ЧАЙ,0.136364,0.600000,0.800000
10,"ЧАЙ, СЫРЫ",СЫРЫ,ЧАЙ,0.295455,0.684211,0.912281
8,"МЕД, ЧАЙ",МЕД,ЧАЙ,0.409091,0.818182,1.090909
30,"ВАФЛИ, МЕД, СЫРЫ","ВАФЛИ, СЫРЫ",МЕД,0.068182,0.600000,1.200000
